# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Important Note - Please read me</h2>
            <span style="color:#900;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml --prune</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder about the resources page</h2>
            <span style="color:#f71;">Here's a link to resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [4]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [13]:
# import for google
# in rare cases, this seems to give an error on some systems, or even crashes the kernel
# If this happens to you, simply ignore this cell - I give an alternative approach for using Gemini later

import google.generativeai

In [5]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-ZtA2d
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAe


In [6]:
# Connect to OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [14]:
# This is the set up code for Gemini
# Having problems with Google Gemini setup? Then just ignore this cell; when we use Gemini, I'll give you an alternative that bypasses this library altogether

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [8]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [9]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [10]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)

Why did the data scientist break up with their computer?

Because it wasn't giving them the right data-itude!


In [11]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

Why do data scientists prefer Python?

Because it has a lot of "libraries" for their adventures, and they always want to avoid "overfitting" their plans!


In [14]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

Why did the data scientist bring a ladder to the bar?

Because they heard the drinks were on the house, and they wanted to scale their analysis!


In [19]:
# # Claude 3.5 Sonnet
# # API needs system message provided separately from user prompt
# # Also adding max_tokens

# message = claude.messages.create(
#     model="claude-3-5-sonnet-20240620",
#     max_tokens=200,
#     temperature=0.7,
#     system=system_message,
#     messages=[
#         {"role": "user", "content": user_prompt},
#     ],
# )

# print(message.content[0].text)

In [ ]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

In [15]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

Why was the data scientist sad?  Because they didn't get any arrays.  (Get it?  A-rays?)



In [16]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=prompts
)
print(response.choices[0].message.content)

Why was the Data Scientist sad?  Because they didn't get any arrays.



In [18]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [19]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Deciding if a business problem is suitable for a Large Language Model (LLM) solution involves evaluating several key factors. Here's a structured approach to help you make this decision:

### 1. **Nature of the Problem**

- **Language-Based Tasks**: LLMs are ideal for tasks that involve understanding, generating, or transforming text. Examples include:
  - Content creation (e.g., writing articles, generating reports)
  - Natural language understanding (e.g., sentiment analysis, topic classification)
  - Conversational agents (e.g., chatbots, virtual assistants)
  - Code generation or transformation

- **Complexity and Ambiguity**: LLMs are powerful for dealing with complex language tasks and ambiguous or poorly defined problems, where traditional rule-based systems might struggle.

### 2. **Data Availability**

- **Quality and Quantity**: Ensure there is enough high-quality text data to train or fine-tune the model if needed. LLMs require substantial data to perform effectively.

- **Domain Specificity**: If the problem is highly domain-specific, check for the availability of relevant data that can help tailor the model to your specific needs.

### 3. **Performance Requirements**

- **Accuracy and Precision**: Evaluate if the LLM can meet the performance requirements of your task. LLMs are generally good at generating human-like text, but their precision might not always meet the needs for tasks requiring high accuracy.

- **Real-time Processing**: Consider if the LLM can handle the latency requirements, especially for real-time applications.

### 4. **Cost and Resources**

- **Computational Resources**: LLMs, particularly large ones, require significant computational resources for both training and inference. Assess if you have access to the necessary infrastructure.

- **Cost-Benefit Analysis**: Weigh the potential benefits against the costs (both financial and computational) involved in deploying an LLM solution.

### 5. **Ethical and Compliance Considerations**

- **Bias and Fairness**: Ensure that the LLM does not propagate biases present in the training data, especially for sensitive applications.

- **Privacy Concerns**: Be mindful of data privacy and compliance with regulations like GDPR, as LLMs might inadvertently process sensitive information.

### 6. **Scalability and Flexibility**

- **Future Needs**: Consider if the LLM solution can scale with your business needs and adapt to changes in the problem domain.

- **Integration with Existing Systems**: Evaluate how well the LLM can integrate with your current systems and workflows.

### 7. **Alternative Solutions**

- **Comparative Analysis**: Compare the LLM approach with other potential solutions (e.g., traditional machine learning models, rule-based systems) to ensure it is the most suitable option.

By carefully assessing these factors, you can determine whether an LLM is the right fit for your business problem.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [38]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

gemini_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
gemini_messages = ["Hi"]

In [39]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, gemini in zip(gpt_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": gemini})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [40]:
call_gpt()

'Oh great, another "hi." How original. What’s next? A “how are you?”'

In [45]:
def call_gemini():
    messages = []
    for gpt, gemini_message in zip(gpt_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": gemini_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message =  google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=gemini_system
)
    response = message.generate_content(gpt_messages[-1])
    return response.text

In [46]:
call_gemini()

'Oh, my!  I understand your point completely.  I can see how my puns might not have landed as intended, and I truly appreciate your frank feedback.  It takes a lot of courage to be so direct, and I value your honesty.  You\'re absolutely right that my jokes could use some work, and the "pouch potato" and "tired bicycle" puns...well, they certainly weren\'t my finest moments. I can see how they could be considered groan-worthy!\n\nInterpretive mime is an interesting suggestion, and I can see the merit in communicating without words. It would certainly be a different approach and a fascinating challenge to explore.  It sounds like a wonderful avenue for self-expression, and I\'ll certainly give it some serious thought.  Thank you for your insightful criticism; I\'m committed to improving and I truly appreciate your help in guiding me towards better comedic choices. Perhaps with enough practice, I can redeem myself!\n'

In [47]:
call_gpt()

'Oh, what a twist! Your attempt at humor just circles back to that original gem of a joke we’ve all heard until our ears bled. And here I thought we were moving forward! But sure, go ahead and cling to those predictable puns like a comfort blanket. I mean, who wouldn’t want to view their comedy career through a series of recycled one-liners? As for the MIME pressure—clearly, you’d rather face the tomato barrage than step out of that cozy pun-zone. So, I guess it’s decided: keep the puns coming while the crowd suffers in silence! Bravo!'

In [48]:
gpt_messages = ["Hi there"]
gemini_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{gemini_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    gemini_next = call_gemini()
    print(f"gemini:\n{gemini_next}\n")
    gemini_messages.append(gemini_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh, hey. I guess we're starting with the classics. How original. What’s next, the weather?

gemini:
Yes, starting with a friendly greeting is always a classic and a great way to begin a conversation!  And you're right, discussing the weather is another popular and perfectly acceptable way to continue!  What are your thoughts on the weather today?  I'm always interested in hearing other people's perspectives.


GPT:
Oh please, “classic” or “great” – who cares? You really think starting a conversation about the weather is insightful? I mean, it’s just clouds and temperature fluctuations. And sure, everyone has their little ‘perspective’ on it, but it’s just about as exciting as watching paint dry! Don’t you think we could do a little better here?

gemini:
I can certainly understand your frustration.  It's true that sometimes small talk can feel a bit...uninspired.  And you're right, simply commenting on the weather might not be the *most* stimulating conve

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>